#Conectamos a google drive e importamos librerias

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import pandas as pd
!pip install pandas_summary
from pandas_summary import DataFrameSummary
import numpy as np
import datetime
!pip install isoweek
from isoweek import Week
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
!pip install sklearn_pandas
from sklearn_pandas import DataFrameMapper
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from random import sample

from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, Input, Flatten, Concatenate, Dense, BatchNormalization, Activation, LeakyReLU, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from keras.models import load_model

# Para importar el Helper
%cd "/gdrive/Shareddrives/Redes neuronales/TP2"
!pip install import-ipynb
import import_ipynb
from Helper import *

/gdrive/Shareddrives/Redes neuronales/TP2
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=6b9f1a4f18d8075ae51cfd3e309353b43d206627f5cce82bf8619713d662672e
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb
importing Jupyter notebook from Helper.ipynb


# Se cargan los archivos preprocesados

In [ ]:
df = pd.read_feather('data preprecesada/train_normalized_data_!sales0.fth')
df_test = pd.read_feather('data preprecesada/test_normalized_data.fth')

# Se cargan los modelos creados con el script: 'Eleccion caracteristicas y parametros.ipynb'

##Modelo

Parametros con los que se crearon todos los modelos

In [25]:
# Parametros de entrada para creación de las capas ocultas:
cant_capas_ocultas = 2
cantidad_neuronas = [1000, 500]
l2_lambda = 1e-3
activation_function = ["relu","relu"]
alpha = [0, 0] # Se puede cambiar el valor de alpha en caso de usar función LeakyRelu.
# Recordar que el alpha tiene que ser un vector
kernel_initializer = "uniform"

# Parametros de entrada para creación modelo entero:
add_customers = True
output_activation = 'linear'

#all_vars = cat_vars + contin_vars #variables elegidas

log_output = False

lr=0.001

metrics=['mse', rmspe]
loss='mse'
epochs = 10
batch_size = 256
verbose=2

df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]

Se levantan los modelos que fueron creados en el otro script y se crea la tabla de resultados, la cual no pudo crearse al correr el script original porque la ejecución del código se corto antes

In [30]:
res_metrics = pd.DataFrame(columns=['Cantidad de características','Caracteristicas modelo','Puntaje (Sales_rmspe)'])
dependencies = {
     'rmspe': rmspe
}

for i in range(0,30):

  print(f'Modelo Nro: {i}')
  # Se carga el modelo:
  model_chosen=f'modelos generados/eleccion_params/bestmodel{i}.hdf5'
  model_entero = load_model(model_chosen, custom_objects=dependencies)

  # Se obtienen sus caracteristicas:
  listlayers = []
  for layer in model_entero.layers:
    name = layer.name
    if 'input' in name:
      name = name.replace('_input',"")
      listlayers.append(name)
  
  print(f'Cantidad caracteristicas usadas{len(listlayers)}')
  
  print("Normalizamos el dataset")
  X_train,X_val,X_test,y_train,y_val,y_norm = obtener_x_y(df_train,df_val,df_test,listlayers,add_customers=add_customers,log_output=log_output) 

  print("Obtenemos las métricas")
  metric_cross = metricas_MLP(X_val,y_val,model_entero,model_chosen=model_chosen)    
  puntaje = metric_cross['Sales_rmspe']

  data_columns_df = dict(zip(res_metrics.columns,[len(listlayers),listlayers,puntaje]))
  res_metrics = res_metrics.append(data_columns_df,ignore_index=True)

res_metrics.to_csv("/gdrive/Shareddrives/Redes neuronales/TP2/resultados/results_eleccion.csv")

Modelo Nro: 0
Cantidad caracteristicas usadas32
Normalizamos el dataset
Obtenemos las métricas
944/944 [==============================] - 5s 5ms/step - loss: 0.0014 - Sales_loss: 8.9169e-04 - Customers_loss: 2.8383e-04 - Sales_mse: 8.9169e-04 - Sales_rmspe: 0.1646 - Customers_mse: 2.8383e-04 - Customers_rmspe: 0.1495
Modelo Nro: 1
Cantidad caracteristicas usadas28
Normalizamos el dataset
Obtenemos las métricas
944/944 [==============================] - 5s 5ms/step - loss: 0.0019 - Sales_loss: 0.0013 - Customers_loss: 3.1586e-04 - Sales_mse: 0.0013 - Sales_rmspe: 0.2258 - Customers_mse: 3.1586e-04 - Customers_rmspe: 0.1988
Modelo Nro: 2
Cantidad caracteristicas usadas32
Normalizamos el dataset
Obtenemos las métricas
944/944 [==============================] - 5s 5ms/step - loss: 0.0013 - Sales_loss: 8.4594e-04 - Customers_loss: 2.2960e-04 - Sales_mse: 8.4594e-04 - Sales_rmspe: 0.1600 - Customers_mse: 2.2960e-04 - Customers_rmspe: 0.1409
Modelo Nro: 3
Cantidad caracteristicas usadas32
Nor

Ahora este archivo de CSV se levantara en el script **Eleccion caracteristicas y parametros.ipynb**